In [1]:
%load_ext autoreload
%autoreload 2

In [192]:
import numpy as np
import proj_funcs as pr
import pandas as pd
import constructor as cr
import optimiser as opt
import random
from scipy.optimize import minimize
from scipy.stats import unitary_group

In [15]:
dim = 2

gm_matrices = pr.get_gellmann_matrices(dim)

bases = pr.get_bases_from_matrices(gm_matrices)
evs = pr.get_eigenvalues_from_matrices(gm_matrices)


In [81]:

basis = np.array(bases[2])
print(evs[2])
print(pd.DataFrame(basis))

proj = pr.construct_projector(basis)
print(pd.DataFrame(proj))

output = proj @ basis

print(output)



[1.0, -1.0]
          0         1
0  0.707107  0.707107
1 -0.707107  0.707107
                    0                   1
0  0.707107+0.000000j  0.707107+0.000000j
1 -0.707107+0.000000j  0.707107+0.000000j
[[-2.23711432e-17+0.j  1.00000000e+00+0.j]
 [-1.00000000e+00+0.j  2.23711432e-17+0.j]]


In [90]:
eta = 1/2
phase = np.pi
BS = ct.BeamSplitter(1,4,eta)
PS = ct.PhaseShifter(1,4,phase)
# print
print(BS.global_U())
print(PS.global_U())

(array([[0],
       [1]]), array([[0, 1]]))
[[ 0.70710678+0.j  0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.70710678+0.j -0.70710678+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]]
(array([[0],
       [1]]), array([[0, 1]]))
[[-1.+1.2246468e-16j  0.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  1.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  1.+0.0000000e+00j
   0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000000e+00j
   1.+0.0000000e+00j]]


# Test constructor class

To test the constructor class and optimiser we will construct a 4-D clements scheme and use an optimiser to find phases.

In [180]:
dim = 4
U = np.eye(dim)
U=unitary_group.rvs(dim)

pattern = [0,2,1,0,2,1]

phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]
phase_bounds
main_guess = [random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

initial_guess = phase_guess + main_guess
print(initial_guess)
args = (dim,U,pattern,None)


res = minimize(cost_func,x0=initial_guess,args =args,method='BFGS')



[5.263754606116256, 3.835513216913355, 2.5227435150139965, 1.2357665246746192, 2.0783843167869054, 0.6495322958272418, 5.69518591549407, 2.232417024273585]


TypeError: can only concatenate list (not "int") to list

In [189]:
'''Test many random matrices'''

dim = 4
pattern = [0,2,1,0,2,1] #  4 mode ideal clements
# pattern = [0,2,4,3,1,0,2,4,3,1,0,2,4,3,1] #  6 mode ideal clements

num_steps = 10

cf_array = []
fid_array = []

for i in range(num_steps):

    print(f'{i+1}/{num_steps}',end = '\r')

    phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

    main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

    initial_guess = phase_guess + main_guess

    U=unitary_group.rvs(dim)

    args = (dim,U,pattern,None)

    res = minimize(opt.cost_func,x0=initial_guess,args =args,method='BFGS')

    cf = opt.cost_func(res.x,*args)

    U_opt = opt.gen_ideal_U_from_param_array(res.x,dim,pattern,None)

    fid = opt.fidelity(U_opt,U)

    fid_array.append(fid)

    cf_array.append(cf)

print(f'Fidelity = {np.mean(fid_array)} ± {np.std(fid_array)}')
print(f'Cost func = {np.mean(cf_array)} ± {np.std(cf_array)}')



Fidelity = 0.9999999999912962 ± 8.383297409314973e-12
Cost func = 6.963223501657948e-11 ± 6.706658230516152e-11


# For Guilia scheme - look for arbitrary N to 2 dimension unitaries using D-raves scheme

Now we need to optimise phases looking only at two rows/columns. For Guilia chip - need a 6 dimensional state to be projected onto 2 basis states - rows are important. 

In [203]:
'''First use ideal interferometer to check the subset optimiser and fidelity metric'''


dim = 4
pattern = [0,2,1,0,2,1] #  4 mode ideal clements
# pattern = [0,2,4,3,1,0,2,4,3,1,0,2,4,3,1] #  6 mode ideal clements

num_steps = 10

inputs = [i for i in range(dim)]
outputs = [2,3]
sub_indices = np.ix_(outputs,inputs)
cf_array = []
fid_array = []

for i in range(num_steps):

    print(f'{i+1}/{num_steps}',end = '\r')

    phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

    main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

    initial_guess = phase_guess + main_guess

    U=unitary_group.rvs(dim)

    args = (dim,U,pattern,True,False,sub_indices)

    res = minimize(opt.subset_cost_func,x0=initial_guess,args =args,method='BFGS')

    cf = opt.subset_cost_func(res.x,*args)

    U_opt = opt.gen_ideal_U_from_param_array(res.x,dim,pattern,True,False,None)
    
    fid = opt.fidelity(U_opt[sub_indices],U[sub_indices])

    fid_array.append(fid)

    cf_array.append(cf)

print(f'Fidelity = {np.mean(fid_array)} ± {np.std(fid_array)}')
print(f'Cost func = {np.mean(cf_array)} ± {np.std(cf_array)}')




Fidelity = 0.9999999999807715 ± 1.5603382260617066e-11
Cost func = 7.691331868884735e-11 ± 6.241344139084554e-11


In [270]:
'''Simulate actual circuit'''

dim = 6

# pattern = [0,2,4,1,3,0,2,4,1,3,0,2,4,1,3]
# pattern = [2,1,3,0,2,4,1,3,0,2,4]
pattern = [0,2,4,1,3,0,2,4,1,3,2]
zero_phases = None #[False,False,False,False,False,False,False,True,False,True,True]
num_steps = 10

inputs = [i for i in range(dim)]
outputs = [2,3]
sub_indices = np.ix_(inputs,outputs)
cf_array = []
fid_array = []

for i in range(num_steps):

    print(f'{i+1}/{num_steps}',end = '\r')

    phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

    main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

    initial_guess = phase_guess + phase_guess + main_guess

    U=unitary_group.rvs(dim)

    args = (dim,U,pattern,True,False,sub_indices,zero_phases)

    res = minimize(opt.subset_cost_func,x0=initial_guess,args =args,method='BFGS')

    cf = opt.subset_cost_func(res.x,*args)

    U_opt = opt.gen_ideal_U_from_param_array(res.x,dim,pattern,True,False,zero_phases)
    
    # fid = opt.fidelity(U_opt[sub_indices],U[sub_indices])

    fid = opt.fidelity_sub(U_opt,U,sub_indices,dim)

    # print(pd.DataFrame(U_opt))
    # print(pd.DataFrame(U[sub_indices]))
    
    
    fid_array.append(fid)

    cf_array.append(cf)

print(f'Fidelity = {np.mean(fid_array)} ± {np.std(fid_array)}')
print(f'Cost func = {np.mean(cf_array)} ± {np.std(cf_array)}')

Fidelity = 0.9856109352671828 ± 0.0073891203360169615
Cost func = 0.17266877679380707 ± 0.08866944403220288


In [282]:
'''Simulate actual circuit'''

dim = 6

# pattern = [0,2,4,1,3,0,2,4,1,3,0,2,4,1,3]
# pattern = [2,1,3,0,2,4,1,3,0,2,4]
pattern = [0,2,4,1,3,0,2,4,1,3,2]
zero_phases = [False,False,False,False,False,False,False,True,False,True,True]
num_steps = 10

inputs = [i for i in range(dim)]
outputs = [2,3]
sub_indices = np.ix_(outputs,inputs)
cf_array = []
fid_array = []

for i in range(num_steps):

    print(f'{i+1}/{num_steps}',end = '\r')

    phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

    main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

    initial_guess = phase_guess + phase_guess + main_guess

    U=unitary_group.rvs(dim)

    args = (dim,U,pattern,True,False,sub_indices,zero_phases)

    res = minimize(opt.subset_cost_func,x0=initial_guess,args =args,method='BFGS')

    cf = opt.subset_cost_func(res.x,*args)

    U_opt = opt.gen_ideal_U_from_param_array(res.x,dim,pattern,True,False,zero_phases)
    
    # fid = opt.fidelity(U_opt[sub_indices],U[sub_indices])

    fid = opt.fidelity_sub(U_opt,U,sub_indices,dim)

    # print(pd.DataFrame(U_opt))
    # print(pd.DataFrame(U[sub_indices]))
    
    
    fid_array.append(fid)

    cf_array.append(cf)

print(f'Fidelity = {np.mean(fid_array)} ± {np.std(fid_array)}')
print(f'Cost func = {np.mean(cf_array)} ± {np.std(cf_array)}')

Fidelity = 0.9995121398767344 ± 0.0006723875220451441
Cost func = 0.005854321479189895 ± 0.008068650264542027


## Try with projective measurements

In [300]:
dim = 6
verbose = True
gm_matrices = pr.get_gellmann_matrices(dim)

bases = pr.get_bases_from_matrices(gm_matrices)


projectors = [pr.construct_projector(basis) for basis in bases]

if verbose:
    for i,(projector,basis) in enumerate(zip(projectors,bases)):

        if i == 10 or i == 21:
            for eigenvector in basis:

                res = projector @ eigenvector
                
                print(np.round(res,2))

            print('\n')

[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]


[0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]




In [322]:


pattern = [0,2,4,1,3,0,2,4,1,3,2]
zero_phases = None#[True for _ in pattern]#[False,False,False,False,False,False,False,True,False,True,True]
num_steps = 10

inputs = [i for i in range(dim)]
outputs = [2,3]
sub_indices = np.ix_(outputs,inputs)
cf_array = []
fid_array = []

output_array = [(0,1),(2,3),(4,5)]
res_dict = dict()
num_steps = 10
final_fid = []
for _ in range(num_steps):
    total_fid_array = []
    for i,(projector,basis) in enumerate(zip(projectors[0:],bases[0:])):

        print(f'{i+1}/{len(bases)}',end = '\r')
        # print(pd.DataFrame(np.round(projector,2)))
        fid_for_basis = []
        # small_d÷ict = dict()
        for j,proj in enumerate(output_array):
            # print(f'Which projector ({proj})')
            
            # small_÷dict[f'{proj}']

            ix = np.ix_(proj,inputs)

            U = projector[ix]

            # print(pd.DataFrame(U))

            phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

            main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

            initial_guess =  phase_guess + phase_guess + main_guess

            args = (dim,U,pattern,True,True,sub_indices,zero_phases)

            res = minimize(opt.subset_cost_func,x0=initial_guess,args =args,method='BFGS')
            # print(res.x[dim:])
            cf = opt.subset_cost_func(res.x,*args)

            U_opt = opt.gen_ideal_U_from_param_array(res.x,dim,pattern,True,True,zero_phases)
            # print(pd.DataFrame(np.round(U_opt,2)))
            fid = opt.fidelity(U_opt[sub_indices],U)
            # fid = opt.fidelity_sub(U_opt,U,sub_indices,dim)
            fid_for_basis.append(fid)

            basis_pair = (basis[2*j],basis[2*j+1])
            
            
            
            '''Check'''
            # vecs = []

            # for eigenstate in basis_pair:

            #     output_vec = U_opt @ eigenstate
                
            #     # print(output_vec)
            #     print(np.round(output_vec[2:4],5))

            #     vecs.append(output_vec)

        total_fid_array.append(np.mean(fid_for_basis))
        print(f'Fidelity for basis {i+1} = {np.mean(fid_for_basis)} ± {np.std(fid_for_basis)}')

    final_fid.append(total_fid_array)


final = [np.mean(l) for l in np.array(final_fid).T]
print(final)
#     print(f'{i+1}/{num_steps}',end = '\r')

#     phase_guess = [2*np.pi*random.uniform(0,1) for i in range(dim)]

#     main_guess = [np.pi*random.uniform(0,1) if i%2 else 2*np.pi*random.uniform(0,1) for i in range(2*len(pattern))]

#     initial_guess = phase_guess + phase_guess + main_guess

#     U=unitary_group.rvs(dim)

#     args = (dim,U,pattern,True,False,sub_indices,zero_phases)

#     res = minimize(opt.subset_cost_func,x0=initial_guess,args =args,method='BFGS')

#     cf = opt.subset_cost_func(res.x,*args)

#     U_opt = opt.gen_ideal_U_from_param_array(res.x,dim,pattern,True,False,zero_phases)
    
#     # fid = opt.fidelity(U_opt[sub_indices],U[sub_indices])

#     fid = opt.fidelity_sub(U_opt,U,sub_indices,dim)

#     # print(pd.DataFrame(U_opt))
#     # print(pd.DataFrame(U[sub_indices]))
    
    
#     fid_array.append(fid)

#     cf_array.append(cf)

# print(f'Fidelity = {np.mean(fid_array)} ± {np.std(fid_array)}')
# print(f'Cost func = {np.mean(cf_array)} ± {np.std(cf_array)}')

Fidelity for basis 1 = 0.9999999999798996 ± 2.4643577734956413e-11
Fidelity for basis 2 = 0.999999999996795 ± 4.177757100395736e-12
Fidelity for basis 3 = 0.999999999987122 ± 8.141251346913114e-12
Fidelity for basis 4 = 0.9999999999513195 ± 6.09339608070987e-11
Fidelity for basis 5 = 0.9999999999326312 ± 8.537999722065252e-11
Fidelity for basis 6 = 0.9999999999731485 ± 2.9386988349822204e-11
Fidelity for basis 7 = 0.9999999999948477 ± 3.0431913659386626e-12
Fidelity for basis 8 = 0.9999999999855763 ± 1.2068505230173222e-11
Fidelity for basis 9 = 0.9999999999941895 ± 5.132545503378562e-12
Fidelity for basis 10 = 0.8333333332799686 ± 0.23570226045959336
Fidelity for basis 11 = 0.9999999999843224 ± 1.682107114836947e-11
Fidelity for basis 12 = 0.999999999986235 ± 7.936250608611129e-12
Fidelity for basis 13 = 0.9999999999886785 ± 9.23851242333606e-12
Fidelity for basis 14 = 0.9999999999970787 ± 1.665634974428747e-12
Fidelity for basis 15 = 0.9999999999878092 ± 9.833401536883126e-12
Fidelit

In [298]:
print(projectors[10])
print(bases[10])

[[ 0.        +0.j  0.70710678+0.j  0.        +0.j  0.70710678+0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j -0.70710678+0.j
   0.        +0.j  0.        +0.j]
 [ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   1.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  1.        +0.j]]
[[ 0.        +0.j  0.70710678+0.j  0.        +0.j  0.70710678+0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j -0.70710678+0.j
   0.        +0.j  0.        +0.j]
 [ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j
   0.        +0.j  0.   